# 78_PCA - iris 2

In [30]:
import pandas as pd
from sklearn.datasets import load_iris
import seaborn as sns
import numpy as np

iris = load_iris()

iris_pd = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_pd['species'] = iris.target
# iris_pd.head()

## StandardScaler 와 PCA 를 묶어서 기억하기

In [31]:
from sklearn.preprocessing import StandardScaler

iris_ss = StandardScaler().fit_transform(iris.data)
# iris_ss[:3]

In [32]:
from sklearn.decomposition import PCA

def get_pca_data(ss_data, n_components = 2):
                # StandardScale 된 데이터
                            # 주성분 개수!
    pca = PCA(n_components=n_components)
    pca.fit(ss_data)
    
    return pca.transform(ss_data), pca

In [33]:
iris_pca, pca = get_pca_data(iris_ss, 2)        # -> pca.transform(ss_data), pca

In [34]:
# PCA (주성분분석) 의 평균, components, 비율
print('pca.mean_', pca.mean_)
print('pca.components_', pca.components_)
print('pca.explained_variance_ratio_',pca.explained_variance_ratio_)

pca.mean_ [-1.69031455e-15 -1.84297022e-15 -1.69864123e-15 -1.40924309e-15]
pca.components_ [[ 0.52106591 -0.26934744  0.5804131   0.56485654]
 [ 0.37741762  0.92329566  0.02449161  0.06694199]]
pca.explained_variance_ratio_ [0.72962445 0.22850762]


In [35]:
# pca.transform(ss_data) 를 Pandas의 DataFrame으로

def get_pd_from_pca(pca_data, cols=['PC1', 'PC2']):
    return pd.DataFrame(pca_data, columns=cols)

In [36]:
iris_pd_pca = get_pd_from_pca(iris_pca)
iris_pd_pca['species'] = iris.target
iris_pd_pca.head()

,PC1,PC2,species
0,-2.264703,0.480027,0
1,-2.080961,-0.674134,0
2,-2.364229,-0.341908,0
3,-2.299384,-0.597395,0
4,-2.389842,0.646835,0


# Ensemble (앙상블) - RandomForest 이용하기

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [38]:
def rf_scores(X, y, cv=5):
    rf = RandomForestClassifier(random_state=13, n_estimators= 100)     # A random forest is a meta estimator
                                # random_stats : 무작위 선정의 케이스를 고정, 가장 좋은(?) 결과물을 나타내기 위해 조정하는 것.
                                                    # n_estimators = : 나무의 갯수 , 기본은 100개
    
    scores_rf = cross_val_score(rf, X, y, scoring='accuracy', cv=cv)    # -> scores를 반환
                                # estimator : estimator object implementing 'fit' // 훈련 모델
                                    # X : data to fit // 훈련용 데이터
                                        # y : the target variable to try to predict // 타겟
                                            # scoring : A str (see model evaluation documentation) or a scorer callable object
                                                                # cv : Determines the cross-validation splitting strategy

    # print(scores_rf)        # cv 개수에 맞는 결과값을 리스트 형식으로 나타냄.
    print('Score :', np.mean(scores_rf))        # scores_rf 의 결과값 5개의 평균(np.mean)을 출력함.
    

In [39]:
%%time
rf_scores(iris_ss, iris.target)

Score : 0.96
CPU times: user 345 ms, sys: 6.83 ms, total: 352 ms
Wall time: 353 ms


In [40]:
%%time
pca_X = iris_pd_pca[['PC1', 'PC2']]
        # iris_pd_pca의 'pc1','pc2' 컬럼을 가져옴.

rf_scores(pca_X, iris.target)

## accuracy가 조금 떨어졌다.

Score : 0.9066666666666666
CPU times: user 340 ms, sys: 6.11 ms, total: 346 ms
Wall time: 347 ms
